In [1]:
import pandas as pd
import joblib
import numpy as np
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score


In [2]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [4]:
train.head(3)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout


In [4]:
duplicates = train[train.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, Marital status, Application mode, Application order, Course, Daytime/evening attendance, Previous qualification, Previous qualification (grade), Nacionality, Mother's qualification, Father's qualification, Mother's occupation, Father's occupation, Admission grade, Displaced, Educational special needs, Debtor, Tuition fees up to date, Gender, Scholarship holder, Age at enrollment, International, Curricular units 1st sem (credited), Curricular units 1st sem (enrolled), Curricular units 1st sem (evaluations), Curricular units 1st sem (approved), Curricular units 1st sem (grade), Curricular units 1st sem (without evaluations), Curricular units 2nd sem (credited), Curricular units 2nd sem (enrolled), Curricular units 2nd sem (evaluations), Curricular units 2nd sem (approved), Curricular units 2nd sem (grade), Curricular units 2nd sem (without evaluations), Unemployment rate, Inflation rate, GDP, Target]
Index: []

[0 rows x 38 columns]


In [6]:
train.isnull().sum()

id                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship 

In [3]:
O_E = OrdinalEncoder()
train[['id','Target']] = O_E.fit_transform(train[['id','Target']])
train.head(2)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2.0
1,1.0,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0.0


In [5]:
train.corr()['Target'].sort_values(ascending=False)

Target                                            1.000000
Curricular units 2nd sem (approved)               0.781452
Curricular units 1st sem (approved)               0.725490
Curricular units 2nd sem (grade)                  0.719036
Curricular units 1st sem (grade)                  0.661355
Tuition fees up to date                           0.415691
Scholarship holder                                0.394124
Curricular units 2nd sem (enrolled)               0.289165
Curricular units 1st sem (enrolled)               0.263657
Curricular units 2nd sem (evaluations)            0.214951
Admission grade                                   0.172880
Course                                            0.154208
Curricular units 1st sem (evaluations)            0.152398
Displaced                                         0.150066
Previous qualification (grade)                    0.138119
Application order                                 0.128394
Daytime/evening attendance                        0.1244

Drop clounms that has no correlation with indipendent variable
1 Educational special needs  
2 International
3 Nacionality
4 Unemployment rate
5 Curricular units 1st sem (credited) 
6 Curricular units 2nd sem (credited)
7 Mother's qualification
8 Previous qualification
9 Father's qualification
10 Father's occupation  
11 Inflation rate 

In [ ]:
train = train.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
test = test.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [50]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

In [82]:
outliers = ((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR)))
print(outliers)

          id  Marital status  Application mode  Application order  Course  \
4      False           False             False              False   False   
7      False           False             False              False   False   
12     False           False             False              False   False   
17     False           False             False              False   False   
22     False           False             False              False   False   
...      ...             ...               ...                ...     ...   
76508  False           False             False              False   False   
76509  False           False             False              False   False   
76511  False           False             False              False   False   
76512  False           False             False              False   False   
76517  False           False             False              False   False   

       Daytime/evening attendance  Previous qualification  \
4             

In [76]:
train = train[~((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR))).any(axis=1)]
print(train)

            id  Marital status  Application mode  Application order  Course  \
4          4.0               1                 1                  2    9500   
7          7.0               1                 1                  1    9773   
12        12.0               1                 1                  1    9853   
17        17.0               1                17                  1    9238   
22        22.0               1                17                  1    9773   
...        ...             ...               ...                ...     ...   
76508  76508.0               1                 1                  1    9238   
76509  76509.0               1                 1                  1    9500   
76511  76511.0               1                17                  1    9773   
76512  76512.0               1                 1                  1    9238   
76517  76517.0               1                 1                  1    9773   

       Daytime/evening attendance  Previous qualifi

In [11]:
print(train.describe())

                 id  Marital status  Application mode  Application order  \
count  76518.000000    76518.000000      76518.000000       76518.000000   
mean   38258.500000        1.111934         16.054419           1.644410   
std    22088.988286        0.441669         16.682337           1.229645   
min        0.000000        1.000000          1.000000           0.000000   
25%    19129.250000        1.000000          1.000000           1.000000   
50%    38258.500000        1.000000         17.000000           1.000000   
75%    57387.750000        1.000000         39.000000           2.000000   
max    76517.000000        6.000000         53.000000           9.000000   

             Course  Daytime/evening attendance  Previous qualification  \
count  76518.000000                76518.000000            76518.000000   
mean    9001.286377                    0.915314                3.658760   
std     1803.438531                    0.278416                8.623774   
min       33.00

In [5]:
model = CatBoostClassifier()

In [4]:
param_grid_catboost = {
    'iterations': [20],
    'depth': [4, 6, 8, 16],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}

In [6]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid_catboost, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

In [7]:
features= train.drop(['id', 'Target'],axis=1)
target=train['Target']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [9]:
grid_search.fit(features, target)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
0:	learn: 0.9890190	total: 5.26s	remaining: 1m 39s
1:	learn: 0.9035134	total: 10.8s	remaining: 1m 37s
2:	learn: 0.8354352	total: 16.1s	remaining: 1m 31s
3:	learn: 0.7826511	total: 16.2s	remaining: 1m 4s
4:	learn: 0.7356961	total: 21.6s	remaining: 1m 4s
5:	learn: 0.6956206	total: 27.3s	remaining: 1m 3s
6:	learn: 0.6615329	total: 32.8s	remaining: 1m
7:	learn: 0.6310801	total: 37.8s	remaining: 56.7s
8:	learn: 0.6047109	total: 43s	remaining: 52.6s
9:	learn: 0.5816827	total: 48.1s	remaining: 48.1s
10:	learn: 0.5611682	total: 53.3s	remaining: 43.6s
11:	learn: 0.5431182	total: 58.4s	remaining: 38.9s
12:	learn: 0.5264029	total: 1m 3s	remaining: 34.2s
13:	learn: 0.5109030	total: 1m 8s	remaining: 29.4s
14:	learn: 0.4972921	total: 1m 13s	remaining: 24.5s
15:	learn: 0.4841771	total: 1m 18s	remaining: 19.7s
16:	learn: 0.4731020	total: 1m 23s	remaining: 14.8s
17:	learn: 0.4635699	total: 1m 28s	remaining: 9.85s
18:	learn: 0.4546754	total: 

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000020B76767CB0>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8, 16], 'iterations': [20],
                         'l2_leaf_reg': [1, 3, 5, 7, 9],
                         'learning_rate': [0.01, 0.05, 0.1]},
             scoring='accuracy', verbose=1)

In [79]:
model.fit(features, target)

Learning rate set to 0.091803
0:	learn: 1.0274498	total: 9.08ms	remaining: 9.07s
1:	learn: 0.9686443	total: 15.7ms	remaining: 7.86s
2:	learn: 0.9205402	total: 23.3ms	remaining: 7.75s
3:	learn: 0.8818584	total: 28.8ms	remaining: 7.16s
4:	learn: 0.8486875	total: 34.3ms	remaining: 6.82s
5:	learn: 0.8218298	total: 40.3ms	remaining: 6.67s
6:	learn: 0.7981656	total: 45.5ms	remaining: 6.46s
7:	learn: 0.7770031	total: 51ms	remaining: 6.32s
8:	learn: 0.7576098	total: 57.5ms	remaining: 6.33s
9:	learn: 0.7403041	total: 62.8ms	remaining: 6.22s
10:	learn: 0.7267301	total: 68.5ms	remaining: 6.16s
11:	learn: 0.7145216	total: 74.4ms	remaining: 6.13s
12:	learn: 0.7030687	total: 80.6ms	remaining: 6.12s
13:	learn: 0.6932569	total: 88.6ms	remaining: 6.24s
14:	learn: 0.6841189	total: 94.7ms	remaining: 6.22s
15:	learn: 0.6758149	total: 99.9ms	remaining: 6.14s
16:	learn: 0.6689807	total: 107ms	remaining: 6.18s
17:	learn: 0.6625734	total: 112ms	remaining: 6.13s
18:	learn: 0.6575840	total: 118ms	remaining: 6.0

In [12]:
testset = test.drop('id' , axis =1)

In [33]:
pred = model.predict(testset)

In [10]:
predt = grid_search.predict(X_test)

In [13]:
pred = grid_search.predict(testset)

In [80]:
predt = model.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, predt)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 86.30%


In [14]:
print("Classification Report:\n", classification_report(y_test,predt ))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.85      0.89      5028
         1.0       0.74      0.67      0.70      3017
         2.0       0.86      0.95      0.90      7259

    accuracy                           0.86     15304
   macro avg       0.85      0.82      0.83     15304
weighted avg       0.86      0.86      0.86     15304



In [15]:
import joblib

In [20]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [16]:
joblib.dump(grid_search, 'model.pkl')

['model.pkl']

In [17]:
pred.shape =(51012,)

In [18]:
predset = pd.DataFrame({'id': test['id'], 'Target': pred})
predset.head(3)

,id,Target
0,76518,0.0
1,76519,2.0
2,76520,2.0


In [19]:
def targetNames(col):
    if col == 0:
        return 'Dropout'
    if col == 1:
        return 'Enrolled'
    if col == 2:
        return 'Graduate'

prediction_df = pd.Series(pred).apply(targetNames)
preddf = pd.DataFrame({'id': test['id'], 'Target': prediction_df})
preddf.head(10)

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled
5,76523,Graduate
6,76524,Graduate
7,76525,Graduate
8,76526,Dropout
9,76527,Graduate


In [21]:
preddf.to_csv('sample_submission.csv', index=False)

In [42]:
joblib.dump(model, 'model.pkl')

['model.pkl']